# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [72]:
import re
import os
import pandas as pd

In [53]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [54]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [55]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [56]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [57]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[1, 61, 77, 1, 1, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['<UNK>', 'b', 'r', '<UNK>', '<UNK>', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [58]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 50
BATCH_SIZE = 128
EMBED_DIM = 48
HIDDEN_SIZE = 16
MAX_LEN = 50
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)#TimeDistributed层的作用就是把Dense层应用到这10个具体的向量上，对每一个向量进行了一个Dense操作
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

6874 21
padding sequences
x_train shape: (6899, 50)
x_test shape: (1987, 50)
trainlabels shape: (6899, 50, 21)
testlabels shape: (1987, 50, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
masking_1 (Masking)          (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 48)            329952    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 32)            8320      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 21)            693       
_________________________________________________________________
crf_1 (CRF)                  (None, 50, 21)            945      

6209/6209 [==============================] - 24s 4ms/step - loss: 2.9196 - crf_viterbi_accuracy: 0.9436 - val_loss: 3.0596 - val_crf_viterbi_accuracy: 0.8938
Epoch 42/50
6209/6209 [==============================] - 19s 3ms/step - loss: 2.9177 - crf_viterbi_accuracy: 0.9454 - val_loss: 3.0559 - val_crf_viterbi_accuracy: 0.8945
Epoch 43/50
6209/6209 [==============================] - 17s 3ms/step - loss: 2.9172 - crf_viterbi_accuracy: 0.9450 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8987
Epoch 44/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9157 - crf_viterbi_accuracy: 0.9464 - val_loss: 3.0587 - val_crf_viterbi_accuracy: 0.8969
Epoch 45/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9149 - crf_viterbi_accuracy: 0.9485 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8962
Epoch 46/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9139 - crf_viterbi_accuracy: 0.9489 - val_loss: 3.0592 - val_crf_viterbi_accuracy:

In [59]:
# save model
model.save("model/ch_ner_model.h5")

In [154]:
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    number = 0
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            number += 1
            if tag_label != "O": # 当前实体tag之前有其他实体     
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
                #result_words.append(('T'+str(number), tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取实体
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                #result_words.append(('T'+str(number), tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取实体
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        number += 1
        result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
        #result_words.append(('T'+str(number),tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取结尾的实体
    return result_words

In [155]:
s = 'abcdefg'
print(s[1:3])
print(s[3:7])

bc
defg


In [156]:
maxlen = 50
result = {}
test_data_path = 'data/chusai_xuanshou/'
for i in range(500):
    test_file = test_data_path+str(i+1000)+'.txt'
    with open(test_file, "r", encoding="utf8") as test:
        sentence = test.read()
    sentences = sentence.split('。')
    y_ner = []

    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    ans = []
    for res in result_words:
        number = res[0]
        #tag_start_end = res[1]
        tag = res[1]
        start = res[2]
        end = res[3]
        word = res[4].replace(' ','_')
        ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start, end-1, word))
        #ans.append(, tag, start,end, "".join(word))
        print('{}\t{} {} {}\t{}'.format('T'+str(len(ans)), tag,start, end-1, word))
    result[i+1000] = ans

T1	DRUG_EFFICACY 138 141	清热解毒
T2	DRUG_EFFICACY 143 146	化湿除带
T3	DRUG_EFFICACY 148 151	祛瘀止痛
T4	DISEASE 160 164	慢性盆腔炎
T1	DISEASE 36 38	糖尿病
T2	SYMPTOM 87 90	月经量少
T3	SYMPTOM 95 100	血虚萎黄后错
T4	SYMPTOM 102 105	血虚萎黄
T5	SYMPTOM 107 110	风湿痹痛
T6	SYMPTOM 112 115	肢体麻木
T7	DISEASE 116 118	糖尿病
T1	PERSON_GROUP 36 41	感冒发热病人
T2	DISEASE 51 53	高血压
T3	DISEASE_GROUP 55 57	心脏病
T4	DISEASE_GROUP 59 60	肝病
T5	DISEASE_GROUP 66 67	肾病
T6	SYMPTOM 90 93	月经紊乱
T7	SYMPTOM 109 110	眩晕
T8	PERSON_GROUP 152 154	过敏者
T9	PERSON_GROUP 158 162	过敏体质者
T10	PERSON_GROUP 192 193	儿童
T11	DRUG_DOSAGE 242 243	颗粒
T12	DRUG_TASTE 245 247	气微香
T13	DRUG_TASTE 249 251	味微苦
T14	SYMPTOM 282 285	烘热汗出
T15	SYMPTOM 287 290	头晕耳鸣
T16	SYMPTOM 297 300	五心烦热
T17	SYMPTOM 302 305	腰背酸痛
T18	SYMPTOM 307 310	大便干燥
T19	SYMPTOM 312 314	心烦易
T20	SYNDROME 387 388	2g
T21	SYNDROME 389 390	*1
T1	DISEASE 55 58	子宫肌瘤
T2	SYNDROME 59 62	气滞血瘀
T3	SYMPTOM 66 69	经期延长
T4	SYMPTOM 71 74	经量过多
T5	SYMPTOM 76 81	经色紫黯有块
T6	SYMPTOM 83 89	小腹或乳房胀痛
T7	DRUG_DOSAGE 145 147	硬胶囊
T8	DRUG_TASTE 160 16

T1	DISEASE 25 26	皮疹
T2	DRUG_EFFICACY 76 79	祛瘀生新
T3	DRUG_EFFICACY 81 84	消肿止痛
T4	SYMPTOM 120 123	小腹疼痛
T1	DISEASE 34 37	__贫血
T2	SYNDROME 108 111	气血不足
T1	FOOD_GROUP 52 53	生冷
T2	FOOD_GROUP 55 56	油腻
T3	DRUG_DOSAGE 85 87	大蜜丸
T4	SYMPTOM 106 109	tr><
T5	SYMPTOM 111 114	r_cl
T6	SYMPTOM 116 119	ss='
T7	SYMPTOM 121 124	olor
T8	SYMPTOM 146 149	h><t
T9	SYMPTOM 151 154	>补肾，
T10	SYMPTOM 156 159	寒，止痛
T11	SYMPTOM 161 164	用于肾经
T12	PERSON_GROUP 165 166	虚寒
T13	DRUG_EFFICACY 167 169	引起的
T14	DRUG_EFFICACY 171 172	腿酸
T15	DRUG_EFFICACY 174 177	，阳痿遗
T16	DRUG_EFFICACY 179 180	，小
T17	SYMPTOM 191 194	_补肾，
T18	SYMPTOM 196 199	寒，止痛
T19	SYMPTOM 201 204	用于肾经
T20	SYMPTOM 206 209	寒引起的
T21	SYMPTOM 210 217	腰腿酸痛，阳痿遗
T22	SYMPTOM 219 222	，小便频
T23	SYMPTOM 224 227	，小腹冷
T24	PERSON_GROUP 391 392	4、
T25	DRUG_DOSAGE 447 450	敏者禁用
T26	DRUG_DOSAGE 452 454	过敏体
T1	DISEASE 33 35	盆腔炎
T2	SYMPTOM 39 42	带下量多
T3	SYMPTOM 44 47	下腹疼痛
T4	DRUG_DOSAGE 55 58	薄膜衣片
T5	FOOD_GROUP 72 73	香，
T6	FOOD_GROUP 75 76	苦。
T7	FOOD_GROUP 78 79	_1
T8	DRUG_DOSAGE 28

T1	DRUG_EFFICACY 1 4	祛瘀生新
T2	DRUG_DOSAGE 70 71	颗粒
T3	DRUG_TASTE 73 74	昧甜
T4	DRUG_TASTE 76 77	微苦
T5	FOOD_GROUP 92 93	生冷
T6	SYNDROME 99 102	气血两虚
T7	SYMPTOM 106 109	月经量少
T8	SYMPTOM 111 114	色淡质稀
T9	SYMPTOM 118 121	头晕心悸
T10	SYMPTOM 123 126	疲乏无力
T11	DISEASE 138 140	高血压
T12	DISEASE_GROUP 142 144	心脏病
T13	DISEASE_GROUP 146 147	肾病
T14	DISEASE 149 151	糖尿病
T15	SYMPTOM 189 191	经量少
T16	PERSON_GROUP 202 206	青春期少女
T17	PERSON_GROUP 208 212	更年期妇女
T18	SYMPTOM 230 231	腹痛
T19	SYMPTOM 234 237	阴道出血
T20	PERSON_GROUP 297 301	过敏体质者
T21	PERSON_GROUP 330 331	儿童
T22	SYNDROME 375 376	血瘀
T23	SYMPTOM 380 383	月经不调
T1	SYMPTOM 10 13	经前腹痛
T2	SYMPTOM 15 18	男子少精
T3	DRUG_EFFICACY 76 79	活血通络
T4	SYMPTOM 105 107	精子少
T5	SYMPTOM 109 112	腰膝冷痛
T1	SYNDROME 100 103	于湿热下
T2	SYMPTOM 105 108	，白带过
T3	DRUG_INGREDIENT 190 191	。川
T4	DRUG_INGREDIENT 193 195	、鸡血
T5	DRUG_EFFICACY 197 199	有活血
T6	DRUG_INGREDIENT 237 238	，狗
T7	DRUG_EFFICACY 240 243	有补益肝
T8	DRUG_DOSAGE 265 267	为硬胶
T9	DRUG_DOSAGE 280 281	色粉
T10	DRUG_TASTE 283 285	；气微
T11	DRUG_TAST

T1	DISEASE 68 73	更年期综合症
T2	SYMPTOM 78 81	潮热汗出
T3	SYMPTOM 83 84	眩晕
T4	SYMPTOM 86 87	耳鸣
T5	SYMPTOM 89 90	失眠
T6	PERSON_GROUP 103 108	徽)有限公司
T7	DISEASE 117 119	，少进
T8	DISEASE_GROUP 121 123	腻。2
T9	DISEASE_GROUP 125 126	感冒
T10	DISEASE 128 130	热病人
T11	DISEASE_GROUP 132 133	宜服
T12	DISEASE_GROUP 135 137	。3.
T13	SYMPTOM 155 158	病等慢性
T14	SYMPTOM 173 174	4.
T15	PERSON_GROUP 241 245	无缓解，应
T16	PERSON_GROUP 274 275	品性
T17	DRUG_DOSAGE 316 319	药品，使
T18	DRUG_TASTE 331 332	师。
T19	DRUG_EFFICACY 362 365	用其他药
T20	SYNDROME 374 376	咨询医
T21	SYMPTOM 389 392	除烦安神
T22	SYMPTOM 394 397	用于肾阴
T23	SYMPTOM 398 401	虚所致的
T24	SYMPTOM 403 406	经前后诸
T25	SYMPTOM 408 411	，症见烘
T1	SYNDROME 25 28	气血两虚
T2	SYMPTOM 32 35	月经不调
T3	SYMPTOM 39 44	月经周期错后
T4	SYMPTOM 46 49	行经量少
T5	SYMPTOM 51 54	精神不振
T6	SYMPTOM 56 59	肢体乏力
T7	SYNDROME 73 76	基本药物
T8	SYMPTOM 91 96	活血调经。用
T9	SYMPTOM 98 101	气血两虚
T10	SYMPTOM 103 106	有血瘀所
T11	SYMPTOM 108 112	的月经不调
T12	DRUG_EFFICACY 163 165	咨询医
T1	DRUG_EFFICACY 35 38	滋阴潜阳
T2	DRUG_EFFICACY 40 43	除烦安神
T3	SYMPTOM 55 5

T1	PERSON_GROUP 69 70	孕妇
T2	DRUG_EFFICACY 83 86	活血化瘀
T3	SYMPTOM 97 100	宿有血块
T4	SYMPTOM 105 108	漏下不止
T5	SYMPTOM 110 113	胎动不安
T6	SYMPTOM 116 119	血瘀经闭
T7	SYMPTOM 121 124	行经腹痛
T8	SYMPTOM 128 130	恶露不
T9	SYMPTOM 148 151	月。__
T1	DRUG_EFFICACY 31 34	滋补气血
T2	DRUG_EFFICACY 36 39	调经舒郁
T3	SYMPTOM 43 46	月经不调
T4	SYMPTOM 48 51	经行腹痛
T5	SYMPTOM 53 56	崩漏下血
T6	SYMPTOM 58 61	赤白带下
T7	SYMPTOM 63 66	贫血衰弱
T8	SYMPTOM 68 71	血晕血脱
T9	SYMPTOM 73 76	产后诸虚
T10	SYMPTOM 78 82	骨蒸潮热_
T11	DRUG_INGREDIENT 174 175	次半
T12	DRUG_EFFICACY 182 188	日2次，温开水
T13	DRUG_INGREDIENT 190 191	服。
T14	DRUG_INGREDIENT 193 194	_方
T15	DRUG_INGREDIENT 196 197	人参
T16	DRUG_INGREDIENT 199 200	白术
T17	DRUG_INGREDIENT 201 202	补气
T18	DRUG_INGREDIENT 204 205	鹿茸
T19	DRUG_EFFICACY 206 209	、鹿角霜
T20	DRUG_INGREDIENT 210 211	壮阳
T21	DRUG_INGREDIENT 215 216	当归
T22	DRUG_EFFICACY 229 232	枸杞滋阴
T23	DRUG_EFFICACY 234 237	肾阿胶补
T1	SYNDROME 37 38	血瘀
T2	SYMPTOM 42 45	月经不调
T3	SYMPTOM 49 52	经水量少
T4	DRUG_EFFICACY 71 74	祛瘀生新
T5	SYMPTOM 78 81	月经量少
T6	SYMPTOM 86 89	经来腹痛
T7	D

...

T1	DRUG_EFFICACY 55 56	祛瘀
T2	SYMPTOM 65 68	恶露不行
T3	SYMPTOM 72 75	小腹冷痛
T4	SYMPTOM 82 85	形寒肢冷
T5	DRUG_EFFICACY 126 127	祛瘀
T6	DRUG_EFFICACY 129 130	止痛
T7	SYMPTOM 136 139	恶露不行
T8	SYMPTOM 143 146	小腹冷痛
T9	SYMPTOM 153 156	形寒肢冷
T10	PERSON_GROUP 157 158	儿童
T11	PERSON_GROUP 159 160	孕妇
T12	SYMPTOM 170 173	恶露不行
T13	SYMPTOM 192 195	阴道流血
T14	SYMPTOM 197 200	月经过多
T15	SYMPTOM 201 203	__热
T16	DRUG_EFFICACY 465 468	纤维蛋白
T17	DRUG_EFFICACY 489 491	解聚，
T18	DRUG_EFFICACY 676 677	膜结
T19	PERSON_GROUP 745 746	化颗
T20	PERSON_GROUP 747 748	粒对
T21	PERSON_GROUP 773 774	萄球
T22	DRUG_EFFICACY 775 778	菌均有很
T23	SYMPTOM 780 783	的抑菌作
T1	DRUG_EFFICACY 1 4	祛瘀生新
T2	SYMPTOM 8 11	月经量少
T3	SYNDROME 35 36	血瘀
T4	SYMPTOM 40 43	月经不调
T5	SYMPTOM 47 50	经水量少
T6	PERSON_GROUP 78 79	孕妇
T7	PERSON_GROUP 105 106	孕妇
T8	DISEASE_GROUP 128 129	__
T1	DRUG_EFFICACY 3 6	养血祛瘀
T2	SYNDROME 8 10	血热证
T3	DRUG_EFFICACY 19 22	收缩子宫
T4	DRUG_EFFICACY 81 84	、促进造
T5	DRUG_EFFICACY 135 138	，抑制醋
T6	SYNDROME 172 173	“血
T7	SYMPTOM 276 279	露不行或
T8	SYMPTOM 281 284	而不畅，

T1	DRUG_EFFICACY 41 44	活血调经
T2	SYNDROME 48 51	气血两虚
T3	SYMPTOM 66 71	月经周期错后
T4	SYMPTOM 73 76	行经量少
T5	SYMPTOM 78 81	精神不振
T6	SYMPTOM 83 86	肢体乏力
T7	DRUG_EFFICACY 95 97	补气血
T8	DRUG_EFFICACY 99 101	调月经
T9	SYNDROME 172 175	气血两虚
T10	SYMPTOM 179 182	月经不调
T11	SYMPTOM 186 191	月经周期错后
T12	SYMPTOM 193 196	行经量少
T13	SYMPTOM 198 201	精神不振
T14	SYMPTOM 203 206	肢体乏力
T1	DRUG_EFFICACY 14 16	补气血
T2	DRUG_EFFICACY 18 20	调月经
T3	DRUG_EFFICACY 52 55	益气养血
T4	DRUG_EFFICACY 57 60	活血调经
T5	SYNDROME 64 67	气血两虚
T6	SYMPTOM 82 87	月经周期错后
T7	SYMPTOM 89 92	行经量少
T8	SYMPTOM 94 97	精神不振
T9	SYMPTOM 99 102	肢体乏力
T10	SYNDROME 134 137	药物同时
T11	SYMPTOM 141 144	能会发生
T12	SYMPTOM 148 153	互作用，详情
T13	SYMPTOM 155 158	咨询医师
T14	SYMPTOM 160 163	药师。_
T15	SYMPTOM 165 169	用于气血两
T1	SYMPTOM 33 36	月经量少
T2	SYMPTOM 41 45	经来腹痛_
T3	SYNDROME 67 68	血瘀
T4	SYMPTOM 81 84	月经不调
T5	DISEASE_GROUP 115 116	基本
T1	DRUG_EFFICACY 3 6	养血祛瘀
T2	SYNDROME 15 17	血热证
T3	SYMPTOM 63 66	行而不畅
T4	SYMPTOM 68 71	夹有血块
T5	DRUG_EFFICACY 133 136	幅度和频
T6	DRUG_EFFICACY 148 151	产妇产后
T7	DRU

T1	DRUG_EFFICACY 1 3	补气血
T2	DRUG_EFFICACY 5 7	调月经
T3	SYMPTOM 11 14	头晕心慌
T4	SYMPTOM 16 19	疲乏无力
T5	SYMPTOM 21 24	月经量少
T6	DRUG_EFFICACY 25 27	补气血
T7	DRUG_EFFICACY 29 31	调月经
T8	DRUG_DOSAGE 76 78	水蜜丸
T9	DRUG_TASTE 80 82	微有香
T10	DRUG_DOSAGE 88 90	微苦。
T11	DRUG_EFFICACY 124 126	_尚不
T12	DRUG_EFFICACY 128 130	确。_
T13	DRUG_EFFICACY 134 136	血，调
T14	DRUG_EFFICACY 138 140	经。_
T15	SYMPTOM 144 147	血，调月
T16	SYMPTOM 149 152	。用于头
T17	SYMPTOM 154 157	心慌，疲
T18	SYMPTOM 162 165	月经量少
T19	DRUG_EFFICACY 166 168	，色淡
T20	SYNDROME 176 179	血，调月
T21	SYMPTOM 183 186	于气血两
T1	DRUG_EFFICACY 5 8	活血调经
T2	SYNDROME 12 15	气血两虚
T3	SYMPTOM 30 35	月经周期错后
T4	SYMPTOM 37 40	行经量少
T5	SYMPTOM 42 45	精神不振
T6	SYMPTOM 47 50	肢体乏力
T7	SYNDROME 54 57	气血两虚
T8	SYMPTOM 61 64	月经不调
T9	SYMPTOM 68 73	月经周期错后
T10	SYMPTOM 75 77	行经量
T11	DRUG_EFFICACY 104 106	经周期
T12	DRUG_EFFICACY 108 110	后、行
T13	DISEASE_GROUP 162 163	。_
T1	SYNDROME 34 35	血瘀
T2	SYMPTOM 39 42	月经不调
T3	SYMPTOM 46 49	经水量少
T4	PERSON_GROUP 50 51	孕妇
T5	DISEASE_GROUP 80 81	__
T6	DRUG_EFFICACY 94

T1	SYNDROME 32 35	湿热下注
T2	SYMPTOM 37 41	赤白带下色
T3	SYNDROME 114 117	。__非
T4	DRUG_EFFICACY 118 119	处方
T5	SYNDROME 147 150	用于湿热
T6	SYMPTOM 151 155	下注止带除
T1	SYNDROME 23 26	气血两虚
T2	SYNDROME 28 29	气滞
T3	SYNDROME 30 31	血瘀
T4	SYMPTOM 35 38	月经不调
T5	SYMPTOM 42 45	月经提前
T6	SYMPTOM 47 50	月经错后
T7	SYMPTOM 52 55	月经量多
T8	SYMPTOM 57 60	神疲乏力
T9	SYMPTOM 62 65	行经腹痛
T10	SYMPTOM 66 70	月经不调_
T11	DRUG_DOSAGE 90 92	__6
T1	SYNDROME 66 69	带__清
T1	DISEASE 43 44	__
T2	DRUG_EFFICACY 68 71	开郁顺气
T3	DRUG_EFFICACY 73 76	调经养血
T4	SYMPTOM 80 83	月经错后
T1	DRUG_EFFICACY 14 17	益气养血
T2	DRUG_EFFICACY 19 22	活血调经
T3	SYNDROME 26 29	气血两虚
T4	SYMPTOM 46 49	月经不调
T5	SYMPTOM 53 58	月经周期错后
T6	SYMPTOM 60 63	行经量少
T7	SYMPTOM 65 68	精神不振
T8	SYMPTOM 70 73	肢体乏力
T9	DRUG_EFFICACY 124 126	补气血
T10	DRUG_EFFICACY 128 130	调月经
T11	SYMPTOM 169 172	月经不调
T12	SYMPTOM 176 181	月经周期错后
T1	SYMPTOM 47 50	月经量少
T2	SYMPTOM 52 56	产后腹痛_
T3	DRUG_EFFICACY 79 82	活血调经
T1	DRUG_EFFICACY 37 43	清热除湿_止带
T2	DRUG_EFFICACY 44 46	清热_
T3	SYNDROME 78 81	湿热下注
T4	SYMPTOM 83 85	赤白带
T1	PER

T1	SYMPTOM 11 14	眩晕耳鸣
T2	SYMPTOM 16 19	烦躁失眠
T3	FOOD_GROUP 121 122	s;
T4	FOOD_GROUP 126 127	__
T5	DISEASE 131 132	食辛
T6	SYMPTOM 143 146	冒时不宜
T7	DISEASE 154 156	月经紊
T8	DISEASE_GROUP 158 160	或其他
T9	DISEASE 162 164	病如：
T10	DISEASE_GROUP 166 170	血压、心脏
T11	DISEASE 172 174	、糖尿
T12	DISEASE 191 193	者，应
T13	PERSON_GROUP 250 252	本品不
T14	PERSON_GROUP 256 260	服用。6、
T15	PERSON_GROUP 288 289	时禁
T16	PERSON_GROUP 290 296	止使用。8、请
T17	PERSON_GROUP 300 304	放在儿童过
T18	PERSON_GROUP 332 333	时禁
T19	DRUG_EFFICACY 382 385	__长春
T20	DRUG_EFFICACY 387 390	平药业有
T21	SYMPTOM 402 405	烦安神。
T22	SYMPTOM 407 414	于更年期潮热汗出
T23	DRUG_EFFICACY 415 418	，眩晕耳
T24	DRUG_EFFICACY 420 423	，烦躁失
T1	SYMPTOM 84 87	。__清
T1	SYNDROME 17 20	气血两虚
T2	SYMPTOM 35 40	月经周期错后
T3	SYMPTOM 42 45	行经量少
T4	SYMPTOM 47 50	精神不振
T5	SYMPTOM 52 55	肢体乏力
T6	SYMPTOM 56 60	行经量少_
T7	SYMPTOM 120 123	__用于
T8	SYMPTOM 127 132	虚所致的月经
T9	DRUG_EFFICACY 163 165	明确。
T10	DRUG_EFFICACY 167 169	_每丸
T1	DRUG_EFFICACY 8 11	活血化瘀
T2	SYMPTOM 22 25	宿有血块
T3	SYMPTOM 30 33	漏下不止
T4	SYMPTO

In [137]:
#“实体类别”、“起始位置”、“结束位置”以空格分隔

In [157]:
for i in range(1000,1500):
    with open('data/submit/%d.ann'%i,'w', encoding='utf-8') as wr:
        wr.write('\n'.join(result[i]))

In [147]:
with open('data/train_data/1.ann', encoding='utf-8') as wr:
    test = wr.read()

In [149]:
test

''

In [129]:
''.join(result[1001])

'T1\tDISEASE 36 39\t糖尿病\nT2\tSYMPTOM 87 91\t月经量少\nT3\tSYMPTOM 95 101\t血虚萎黄后错\nT4\tSYMPTOM 102 106\t血虚萎黄\nT5\tSYMPTOM 107 111\t风湿痹痛\nT6\tSYMPTOM 112 116\t肢体麻木\nT7\tDISEASE 116 119\t糖尿病\n'

In [116]:
#for i in range(1000,1500):
#    pd.DataFrame(result[i]).to_csv('data/submit/%d.ann'%i,
#                                      sep='\t',
#                                      header = None,
#                                      index = 0,
#                                      encoding = 'utf-8')